### How to run: Run All directly
### When making changes on the parameters, make sure change it as well on prediction file
### Note: Don't escape when making training and ensure the model has been saved correctly


###

### Import necessary libraries.

In [1]:
import torch
from torch import nn, div, square, norm
from torch.nn import functional as F

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

### Load data and set device

In [2]:
def set_up_environment():  
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    return device

def create_user_item_ratingMatrix(df,num_users,num_items):
    user_item_ratingMatrix = torch.zeros((num_users, num_items))
    for row in df.itertuples():
        user_item_ratingMatrix[row[1]-1, row[2]-1] = row[3]
    return user_item_ratingMatrix

def load_data(Upath,Mpath,Rpath):
    num_users = pd.read_csv(Upath,delimiter="::",header=None,engine='python')[0].max()
    num_items = pd.read_csv(Mpath,delimiter="::",header=None,engine='python')[0].max()
    df_ratings = pd.read_csv(Rpath, sep='::', names=['user_id', 'MovieID', 'rating', 'timestamp'])
    df_movies = pd.read_csv(Mpath, sep="::", header=None, names=["MovieID", "Title", "Genres"], engine="python")
    user_item_ratingMatrix = create_user_item_ratingMatrix(df_ratings,num_users,num_items)
    return user_item_ratingMatrix, num_users, num_items, df_ratings, df_movies


### Convert a list of items into a PyTorch LongTensor,

In [3]:
def collate_fn(batch):
    return torch.LongTensor(batch)

### Turn the train,test,whole dataset into a DataLoader
### Code Reference: https://github.com/tuanio/AutoRec

In [4]:
# Split data into train and test
def Create_train_test(num_items,batch_size=512,num_workers=2):
    
    train_items,test_items = train_test_split(torch.arange(num_items),
                                           test_size=0.2,
                                           random_state=12)
    
    train_dl = DataLoader(train_items,shuffle=True,num_workers=num_workers,batch_size=batch_size,drop_last=True,collate_fn=collate_fn)
    test_dl = DataLoader(test_items, shuffle=False,num_workers=num_workers,batch_size=batch_size,collate_fn=collate_fn)
    whole_dl = DataLoader(torch.arange(num_items), shuffle=False,num_workers=1,batch_size=num_items,collate_fn=collate_fn)
    
    return train_dl,test_dl,whole_dl

### AutoRec model
### Reference: https://github.com/tuanio/AutoRec
### Paper reference: http://users.cecs.anu.edu.au/~u5098633/papers/www15.pdf

In [5]:
class AutoRec(nn.Module):
    def __init__(self, visibleDimensions, hiddenDimensions, learningRate):
        super().__init__()
        self.learningRate = learningRate
        self.weight1 = nn.Parameter(torch.randn(visibleDimensions, hiddenDimensions))
        self.weight2 = nn.Parameter(torch.randn(hiddenDimensions, visibleDimensions))
        self.bias1 = nn.Parameter(torch.randn(hiddenDimensions))
        self.bias2 = nn.Parameter(torch.randn(visibleDimensions))
    
    def regularization(self):
        return div(self.learningRate, 2) * (square(norm(self.weight1)) + square(norm(self.weight2)))
    
    def forward(self, data):
        encoder = self.weight2.matmul(data.T).T + self.bias1
        return self.weight1.matmul(encoder.sigmoid().T).T + self.bias2

#### Evaluation function, calculate RMSE
#### Code Reference: https://github.com/tuanio/AutoRec
#### Apply the same method to make 0 rating as -1, then put into the training, and make -1 rating as 0
#### Return RMSE

In [6]:
def eval_epoch(model, test_set, criterion):
    model.eval()
    truth = []
    predict = []
    loss = []
    with torch.no_grad():
        for _ ,items_idx in enumerate(test_set):
            ratings = user_item_ratingMatrix[:, items_idx].squeeze().permute(1,0).to(device)
            ratings[ratings==0] = -1
            ratings_prediction = model(ratings)
            ratings_prediction[ratings == -1] = 0
            ratings[ratings == -1] = 0
            truth.append(ratings)
            predict.append(ratings_prediction * torch.sign(ratings))       
            single_loss = criterion(ratings, ratings_prediction * torch.sign(ratings)) + model.regularization()
            loss.append(single_loss.item())

    rmse = torch.Tensor([torch.sqrt(square(ratings - ratings_prediction).sum() / torch.sign(ratings).sum())
                            for ratings, ratings_prediction in zip(truth, predict)]).mean().item()
    return loss, rmse

#### Training model
#### Code Reference: https://github.com/tuanio/AutoRec
#### Difference from the original code: make 0 rating as -1, then put into the training, and make -1 rating as 0
#### Aim: to avoid the loss of 0 rating
#### Paper reference: http://users.cecs.anu.edu.au/~u5098633/papers/www15.pdf

In [7]:
def training(model,train_set,user_item_ratingMatrix,optimizer,criterion):
    lossList = []
    for _ , item_idx in enumerate(train_set):
        ratings = user_item_ratingMatrix[:,item_idx].squeeze().permute(1,0).to(device)
        ratings[ratings == 0] = -1
        predict_ratings = model(ratings)
        predict_ratings[ratings == -1] = 0
        loss = criterion(ratings, predict_ratings * torch.sign(ratings)) + model.regularization()       
        lossList.append(loss.item())        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return lossList

#### Train the model, if met the lowest RMSE will save the model
#### Code Reference: https://github.com/tuanio/AutoRec

In [8]:
print("Setting up the environment...")
device = set_up_environment()
print("Loading data...")
user_item_ratingMatrix, num_users, num_items, df_ratings, df_movies = load_data(Upath="./ml-1m/users.dat",Mpath="./ml-1m/movies.dat",Rpath="./ml-1m/ratings.dat")

print("Creating train and test sets...")
train_set, test_set, whole_set = Create_train_test(num_items=num_items,batch_size=32)

def Train_and_Save():
    print("Creating model...")
    model = AutoRec(visibleDimensions=num_users, hiddenDimensions=500, learningRate=0.0001).to(device)

    print("Creating optimizer and criterion...")
    optimiser = torch.optim.Adam(model.parameters(), lr=0.012, weight_decay=1e-5)
    criterion = nn.MSELoss().to(device)

    print("Creating data loaders...")
    max_epochs = 100
    losses = []
    eval_losses = []
    eval_rmse = []
    min_rmse = 1000
    
    print("Training model...")
    for epoch_idx in range(max_epochs):
        print("=" * 10 + f"Epoch: {epoch_idx}" + "=" * 10)
        epoch_loss = training(model,train_set,user_item_ratingMatrix,optimiser,criterion)
        evaluation_loss, rmse = eval_epoch(model, test_set, criterion)
        losses.extend(epoch_loss)
        eval_losses.extend(evaluation_loss)
        if rmse < min_rmse:
            print("Saving model...")
            min_rmse = rmse
            
            # change the path to your own path, and name the model as you wish
            torch.save(model.state_dict(), './model/AutoRec.pth')
        eval_rmse.append(rmse)
        print("Epoch Loss: ", losses[-1])
        print("Evaluation Loss: ", eval_losses[-1])
        print("RMSE: ", eval_rmse[-1])
    return losses, eval_losses, eval_rmse

Setting up the environment...
Loading data...


/Users/vincentzhao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Creating train and test sets...


In [9]:
losses, eval_losses,eval_rmse = Train_and_Save()

Creating model...
Creating optimizer and criterion...
Creating data loaders...
Training model...
==========Epoch: 0==========
Saving model...
Epoch Loss:  83.09021759033203
Evaluation Loss:  80.69125366210938
RMSE:  2.756258964538574
==========Epoch: 1==========
Saving model...
Epoch Loss:  22.46796417236328
Evaluation Loss:  21.197832107543945
RMSE:  1.4833197593688965
==========Epoch: 2==========
Saving model...
Epoch Loss:  7.12447452545166
Evaluation Loss:  6.072370529174805
RMSE:  1.1608784198760986
==========Epoch: 3==========
Saving model...
Epoch Loss:  3.02280855178833
Evaluation Loss:  2.0235443115234375
RMSE:  1.0861060619354248
==========Epoch: 4==========
Saving model...
Epoch Loss:  1.7949788570404053
Evaluation Loss:  0.8100163340568542
RMSE:  1.0845259428024292
==========Epoch: 5==========
Epoch Loss:  1.377124309539795
Evaluation Loss:  0.3910427391529083
RMSE:  1.0932574272155762
==========Epoch: 6==========
Epoch Loss:  1.2027153968811035
Evaluation Loss:  0.22496247